In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
training=pd.read_csv('/kaggle/input/fake-news/train.csv')

In [ ]:
training.head()

In [ ]:
training.drop('id', axis=1, inplace=True)

In [ ]:
training.head()

In [ ]:
len(training)

In [ ]:
training.info()

In [ ]:
training.drop_duplicates(inplace=True)

In [ ]:
training.reset_index()

In [ ]:
training.info()

In [ ]:
training.drop(['text', 'author'], axis=1, inplace=True)

In [ ]:
training.dropna(inplace=True)

In [ ]:
training.info()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train, y_train=training.iloc[: ,0], training.iloc[:,1]

In [ ]:
vocab_size=1000

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [ ]:
lem=WordNetLemmatizer()

In [ ]:
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [ ]:
corpus=[]
for i in range(len(x_train)):
    words=re.sub('[^a-zA-Z]', ' ', x_train.iloc[i])
    words=nltk.word_tokenize(words)
    words=[lem.lemmatize(word.lower()) for word in words if word not in stopwords.words('english')]
    corpus.append(' '.join(words))

In [ ]:
input_sequence=[one_hot(words,vocab_size) for words in corpus]

In [ ]:
max_len=max(len(x) for x in input_sequence)

In [ ]:
padded_sequence=pad_sequences(input_sequence, padding='pre', maxlen=max_len)

In [ ]:
padded_sequence

In [ ]:
model=Sequential([
    Embedding(input_dim=vocab_size, output_dim=45, input_length=max_len),
    Bidirectional(LSTM(units=100)),
    Dense(units=1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(padded_sequence, y_train, test_size=0.3)

In [ ]:
model.fit(x_train, y_train, epochs=50)

In [ ]:
lo=model.history.history
plt.plot(lo['accuracy'])

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))